#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


In [3]:
file_path_list

['/home/workspace/event_data/2018-11-27-events.csv',
 '/home/workspace/event_data/2018-11-04-events.csv',
 '/home/workspace/event_data/2018-11-07-events.csv',
 '/home/workspace/event_data/2018-11-09-events.csv',
 '/home/workspace/event_data/2018-11-19-events.csv',
 '/home/workspace/event_data/2018-11-05-events.csv',
 '/home/workspace/event_data/2018-11-22-events.csv',
 '/home/workspace/event_data/2018-11-16-events.csv',
 '/home/workspace/event_data/2018-11-26-events.csv',
 '/home/workspace/event_data/2018-11-24-events.csv',
 '/home/workspace/event_data/2018-11-29-events.csv',
 '/home/workspace/event_data/2018-11-15-events.csv',
 '/home/workspace/event_data/2018-11-20-events.csv',
 '/home/workspace/event_data/2018-11-06-events.csv',
 '/home/workspace/event_data/2018-11-18-events.csv',
 '/home/workspace/event_data/2018-11-21-events.csv',
 '/home/workspace/event_data/2018-11-10-events.csv',
 '/home/workspace/event_data/2018-11-23-events.csv',
 '/home/workspace/event_data/2018-11-02-events

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster()
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print("Unable to connect to a cluster")

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS myspace
    with REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as e:
    print("Uanble to create a Keyspace")
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('myspace')
except Exception as e:
    print(e)

### Read CSV file

In [9]:
file = 'event_datafile_new.csv'
df = pd.read_csv(file)
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


### Query 1
```SELECT artist, song, length FROM sessions WHERE session_id=338 AND item_in_session=4```
- Get artist's name, song's title and length of a song, given particular `session_id` and `item_in_session`.
- To efficiently query this, we need to make a seperate table with `sessionId` as partition_key and to give uniqueness to
  each row we will have a primary key of `(session_id, item_in_session)` since a user can have multiple items in a
  single session.

In [11]:
query1 = "SELECT artist, song, length FROM sessions WHERE session_id=338 AND item_in_session=4"
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS sessions
    ( 
      session_id int,
      item_in_session int,
      artist text,
      song text,
      length float,
      PRIMARY KEY (session_id,item_in_session)
    )""")
except Exception as e:
    print(e)

In [12]:
for i in range(0, df.shape[0]):
    query = "INSERT INTO sessions (session_id, item_in_session, artist, song, length)"
    query = query + "VALUES (%s, %s, %s, %s, %s)"
    try:
        session.execute(query, df.loc[i,['sessionId','itemInSession','artist','song','length']].tolist())
    except Exception as e:
        print('unable to insert')

In [13]:
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2
```SELECT artist, song, first_name, last_name FROM user_sessions WHERE user_id=10 AND session_id=182```
- Get artist's name, song title, user's first name and last name, given `user_id` and `session_id`
- Also sort tuples according to itemInSession
- To achieve this, we partion data with `(user_id, session_id)` so that song songs listened by user reside in same node.
- We add a additional clustering column `item_in_session` to the primary key, which gives uniqueness as well as it sorts data.
- Our final primary key is `((user_id, session_id),item_in_session)`

In [14]:
query2 = "SELECT artist, song, first_name, last_name FROM user_sessions WHERE user_id=10 AND session_id=182"
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS user_sessions
    ( user_id int,
      session_id int,
      item_in_session int,
      artist text,
      song text,
      first_name text,
      last_name text,
      PRIMARY KEY ((user_id,session_id), item_in_session)
    )""")
except Exception as e:
    print(e)  

In [15]:
for i in range(0, df.shape[0]):
    query = "INSERT INTO user_sessions (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
    query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
    try:
        session.execute(query, df.loc[i,['userId','sessionId','itemInSession','artist','song','firstName','lastName']].tolist())
    except Exception as e:
        print('unable to insert')

In [16]:
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
for row in rows:
    print(row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3
```SELECT first_name, last_name FROM songs_listened WHERE song='All Hands Against His Own'```
- Get user's first name and last name who listened to a particular `song`.
- We partition data on the basis of song, so tuples belonging to each song stays at one place

In [20]:
query3 = "SELECT first_name, last_name FROM songs_listened WHERE song='All Hands Against His Own'"
try:
    session.execute("""
    CREATE TABLE IF NOT EXISTS songs_listened
    ( song text,
      user_id int,
      first_name text,
      last_name text,
      PRIMARY KEY (song, user_id)
    )""")
except Exception as e:
    print(e)                    

In [21]:
for i in range(0, df.shape[0]):
    query = "INSERT INTO songs_listened (song, user_id, first_name, last_name)"
    query = query + "VALUES (%s, %s, %s, %s)"
    try:
        session.execute(query, df.loc[i,['song','userId','firstName','lastName']].tolist())
    except Exception as e:
        print('unable to insert')

In [22]:
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
for row in rows:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [23]:
try:
    session.execute('drop table songs_listened')
except Exception as e:
    print(e)
try:
    session.execute('drop table user_sessions')
except Exception as e:
    print(e)
try:
    session.execute('drop table sessions')
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [86]:
session.shutdown()
cluster.shutdown()